In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.io import read_image
from torchvision.transforms.functional import resize
from PIL import Image
import os
import matplotlib.pyplot as plt

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 1. Build an image segmentation model using pytorch

In [ ]:
# Custom Dataset class
class LungSegmentationDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(image_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.images[idx])
        mask_path = os.path.join(self.mask_dir, self.images[idx].replace('.jpg', '_mask.jpg'))
        image = Image.open(img_path).convert("L")
        mask = Image.open(mask_path).convert("L")

        if self.transform is not None:
            image = self.transform(image)
            mask = self.transform(mask)

        return image, mask

# Define the U-Net architecture
class UNet(nn.Module):
    def __init__(self, in_channels=1):
        super(UNet, self).__init__()

        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.middle = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.decoder = nn.Sequential(
            nn.Conv2d(128, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(64, 1, kernel_size=2, stride=2)
        )

    def forward(self, x):
        x1 = self.encoder(x)
        x2 = self.middle(x1)
        x3 = self.decoder(x2)

        return x3

# Compose the transformations
transform = transforms.Compose([transforms.ToTensor(), transforms.Resize((256, 256))])

# Create datasets and dataloaders
train_dataset = LungSegmentationDataset(image_dir='/content/drive/MyDrive/Lung_segmentation/Train/Images',
                                        mask_dir='/content/drive/MyDrive/Lung_segmentation/Train/Masks',
                                        transform=transform)

test_dataset = LungSegmentationDataset(image_dir='/content/drive/MyDrive/Lung_segmentation/Test/Images',
                                       mask_dir='/content/drive/MyDrive/Lung_segmentation/Test/Masks',
                                       transform=transform)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)


In [ ]:
sample = next(iter(train_loader))
image, target = sample

# Print the shape of the image and target
print("Image shape:", image.shape)
print("Target shape:", target.shape)

Image shape: torch.Size([4, 1, 256, 256])
Target shape: torch.Size([4, 1, 256, 256])


# 2. Train your model using [lung segmentation datasets](https://github.com/YoushanZhang/Lung_Segmentation)

In [ ]:
# Initialize the model, loss function, and optimizer
model = UNet(in_channels=1)  # Assuming your input images are single-channel
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        inputs, targets = batch

        optimizer.zero_grad()
        outputs = model(inputs)

        targets_resized = transforms.Resize((outputs.shape[2], outputs.shape[3]))(targets)

        loss = criterion(outputs, targets_resized)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

# Save the trained model
torch.save(model.state_dict(), '/content/drive/MyDrive/Lung_segmentation/segmentation_model1.pth')

Epoch [1/50], Loss: 0.45907673239707947
Epoch [2/50], Loss: 0.39476674795150757
Epoch [3/50], Loss: 0.4474101960659027
Epoch [4/50], Loss: 0.3101367950439453
Epoch [5/50], Loss: 0.33101320266723633
Epoch [6/50], Loss: 0.3058161735534668
Epoch [7/50], Loss: 0.30677154660224915
Epoch [8/50], Loss: 0.26566416025161743
Epoch [9/50], Loss: 0.2865278124809265
Epoch [10/50], Loss: 0.29995888471603394
Epoch [11/50], Loss: 0.29523706436157227
Epoch [12/50], Loss: 0.23977160453796387
Epoch [13/50], Loss: 0.3756483793258667
Epoch [14/50], Loss: 0.2244366556406021
Epoch [15/50], Loss: 0.27847522497177124
Epoch [16/50], Loss: 0.1903161108493805
Epoch [17/50], Loss: 0.19471152126789093
Epoch [18/50], Loss: 0.1909979283809662
Epoch [19/50], Loss: 0.2529662251472473
Epoch [20/50], Loss: 0.2113085836172104
Epoch [21/50], Loss: 0.21344242990016937
Epoch [22/50], Loss: 0.17239150404930115
Epoch [23/50], Loss: 0.1683899611234665
Epoch [24/50], Loss: 0.1970588117837906
Epoch [25/50], Loss: 0.22122073173522

# 3.Evaluate your model using the test images

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np

# Load the trained model
model = UNet(in_channels=1)
model.load_state_dict(torch.load('/content/drive/MyDrive/Lung_segmentation/segmentation_model1.pth'))
model.eval()

# Evaluate on the test set
IoU_values = []
Dice_values = []

with torch.no_grad():
    for batch in test_loader:
        inputs, targets = batch
        outputs = model(inputs)

        # Resize target to match the output size
        targets_resized = F.interpolate(targets, size=(outputs.shape[2], outputs.shape[3]), mode='nearest')

        # Convert predictions and targets to binary masks
        predictions = torch.sigmoid(outputs) > 0.5
        targets_binary = targets_resized > 0.5

        # Calculate IoU and Dice
        intersection = (predictions & targets_binary).sum().item()
        union = (predictions | targets_binary).sum().item()
        dice = (2.0 * intersection) / (predictions.sum().item() + targets_binary.sum().item())

        IoU = intersection / union
        IoU_values.append(IoU)
        Dice_values.append(dice)


# 4. Your IoU score should be higher than 0.85

In [ ]:
# Calculate the average IoU and Dice
average_IoU = np.mean(IoU_values)
average_Dice = np.mean(Dice_values)

print(f'Average IoU: {average_IoU}')
print(f'Average Dice: {average_Dice}')

Average IoU: 0.8593397793014086
Average Dice: 0.9381251499711834


# 5. Write a 2-page report using LaTex and upload your paper to ResearchGate or Arxiv, and put your paper link here.


In [ ]:
https://www.researchgate.net/publication/376267327_Lung_Segmentation_from_CT_Scans_using_Neural_Networks
https://github.com/Shashi1119/AIM-5001/blob/main/segmentation_model1.pth

# 6. Grading rubric

(1). Code ------- 20 points (you also need to upload your final model as a pt file, and add paper link)

(2). Grammer ---- 20 points

(3). Introduction & related work --- 10 points

(4). Method  ---- 20 points

(5). Results ---- 20 points

     > = 0.85 -->10 points
     < 0.8  --> 0 points
     >= 0.8 & < 0.85  --> 2 point/0.01 higher
     

(6). Discussion - 10 points